In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

import pickle 
#import mglearn
import time


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer



from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

C:\Users\Aditya\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
## importing classifiers 
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from xgboost.sklearn import XGBClassifier

C:\Users\Aditya\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
news = pd.read_csv('E:/Datathon 2019/task-1/task-1.csv',encoding='utf-8')


In [4]:
#news['news_type'] = news['news_type'].apply(lambda x: 1 if x=='propaganda' else 0)
#news_dummies = pd.get_dummies(news)

In [5]:
news = news.dropna()

In [6]:
news['news_number'] = news['news_number'].astype(int)

In [7]:
news.dtypes

news_text      object
news_number     int32
news_type      object
dtype: object

In [8]:
news.isnull().sum()

news_text      0
news_number    0
news_type      0
dtype: int64

In [9]:
news.shape

(35955, 3)

In [10]:
news.head()
print(news.isnull().sum())

news_text      0
news_number    0
news_type      0
dtype: int64


In [11]:
news=news.set_index(["news_number"])

In [12]:
train_x,test_x,train_y, test_y = train_test_split(news['news_text'],news['news_type'],
                              test_size = 0.3,
                              random_state = 100)

In [13]:

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(25168,)
(25168,)
(10787,)
(10787,)


In [14]:
type(train_x)

pandas.core.series.Series

### Cleaning Test Data

In [15]:
test_data =pd.read_csv('test_task1.csv',encoding = 'utf-8')

In [16]:
test_data.head()

,news_text,news_number,news_type
0,Chicago police are seeking the publicâ€™s help...,100013.0,?
1,Moment of silence at Palestine Pavilion during...,100015.0,?
2,The Organization of Islamic Cooperation (OIC) ...,100024.0,?
3,The chairman of the House Oversight and Govern...,100031.0,?
4,A judge in the United Kingdom has sentenced a ...,100040.0,?


In [17]:
test_data = test_data.dropna()


In [18]:
test_data['news_number']= test_data['news_number'].astype(int)

In [19]:
test_data =test_data.set_index(['news_number'])

In [20]:
test_data = test_data.drop(['news_type'],axis =1)

In [21]:
test_data.head()

,news_text
news_number,
100013,Chicago police are seeking the publicâ€™s help...
100015,Moment of silence at Palestine Pavilion during...
100024,The Organization of Islamic Cooperation (OIC) ...
100031,The chairman of the House Oversight and Govern...
100040,A judge in the United Kingdom has sentenced a ...


In [32]:
test_data.shape

(10150, 1)

In [ ]:
## selected final model is Multinomial NB

## TF IDF Vector

In [22]:
tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
tf.fit(train_x)
tdf_train = tf.transform(train_x)
tdf_test = tf.transform(test_x)

In [23]:
tdf_test_data = tf.transform(test_data)

In [24]:
print(tdf_train.shape)
print(tdf_test.shape)
print(tdf_test_data.shape)

(25168, 177939)
(10787, 177939)
(1, 177939)


In [ ]:
model = MultinomialNB()
model.fit(tdf_train, train_y)
pred_test = model.predict(tdf_test)
pred = model.predict(test_data)
tn, fp, fn, tp = confusion_matrix(test_y, pred_test).ravel()
accuracy = accuracy_score(test_y, pred_test) * 100
sensitivity = tp / (tp + fn) * 100
specificity = tn / (tn + fp) * 100
f1 = (2*sensitivity*specificity)/(sensitivity+specificity)


In [16]:
news_tfidf_df = pd.DataFrame(columns=['Decision Tree', 'Random Forest', 'AdaBoost', 'KNN','PassiveAggressive','MultinomialNB','MLPClassifier','svm','XGBClassifier'], 
                       index=['True Positive', 'True Negative', 'False Positive', 'False Negative', 'Accuracy', 'Sensitivity', 'Specificity','F1 score'])

In [17]:
count = 0
while count<9:
    if count == 0:
        model = DecisionTreeClassifier(random_state = 100, max_depth = 5)
    elif count == 1:
        model = RandomForestClassifier(random_state=100, n_estimators=300)
    elif count == 2:
        model = AdaBoostClassifier(random_state=100,n_estimators=300)
    elif count == 3:
        model = KNeighborsClassifier(n_neighbors = 5)
    elif count == 4:
        model = PassiveAggressiveClassifier(n_iter=50)
    elif count == 5:
        model = MultinomialNB()
    elif count == 6:
        model = MLPClassifier(hidden_layer_sizes=(30))
    elif count == 7:
        model = SVC()
    else:
        model = XGBClassifier()
    model.fit(tdf_train, train_y)
    pred_test = model.predict(tdf_test)
    tn, fp, fn, tp = confusion_matrix(test_y, pred_test).ravel()
    accuracy = accuracy_score(test_y, pred_test) * 100
    sensitivity = tp / (tp + fn) * 100
    specificity = tn / (tn + fp) * 100
    f1 = (2*sensitivity*specificity)/(sensitivity+specificity)
    news_tfidf_df.iloc[:,count] = [tp,tn,fp,fn,accuracy,sensitivity,specificity,f1]
    count = count + 1
news_tfidf_df

C:\Users\Aditya\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\Aditya\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,Decision Tree,Random Forest,AdaBoost,KNN,PassiveAggressive,MultinomialNB,MLPClassifier,svm,XGBClassifier
True Positive,376.000000,245.000000,888.000000,205.000000,862.000000,967.000000,803.000000,517.000000,622.000000
True Negative,9584.000000,9589.000000,9433.000000,9571.000000,9399.000000,8937.000000,9513.000000,9569.000000,9550.000000
False Positive,9.000000,4.000000,160.000000,22.000000,194.000000,656.000000,80.000000,24.000000,43.000000
False Negative,818.000000,949.000000,306.000000,989.000000,332.000000,227.000000,391.000000,677.000000,572.000000
Accuracy,92.333364,91.165292,95.679985,90.627607,95.123760,91.814221,95.633633,93.501437,94.298693
Sensitivity,31.490787,20.519263,74.371859,17.169179,72.194305,80.988275,67.252931,43.299832,52.093802
Specificity,99.906182,99.958303,98.332117,99.770666,97.977692,93.161680,99.166059,99.749818,99.551756
F1 score,47.887319,34.049006,84.689913,29.296780,83.132730,86.649506,80.149605,60.386731,68.396721


In [18]:
news_tfidf_df.to_csv("tf_idf.csv")